# Patient Allocation

This script reads data from DBRaw and allocates each patient into one of the following categories:

1. DBClean -> Contains data that doesn't need to be completed at all
2. DBCompleted -> Contains data that underwent completion (due to exam periodicity)
4. DBGarbage -> All of the rest

The main concern is to **outline the reasons for why patients end up in DBGarbage**, in order to be able to quantify the reasons for discarting patients.

I'ts importan to to note that, in order to increase the number of patients in DBCompleted, we will start the analysis for each patient from their **first complete row**, e.g. the first row that has all the data filled in (no missing values). If the patient has no complete rows, we will send them to DBGarbage.

Below we declare some variables that will be used to attribute patients to a DB subset. These values can be tightened or loosened up in order to increase the number of patients in DBCompleted.

In [48]:
x = int(input("Digite a tolerancia de mêses"))
y = int(input("Digite o periodo de tolerancia para outros exames"))
z =int(input("Digite as vezes que pode quebrar a tolerancia de meses"))

In [1]:
MAX_SEQUENCE_BREAK_TOLERANCE = 2 # Maximum gap, in months, that a patient can have between exam appearances.
MAX_PERIOD_TOLERANCE_FOR_MONTHLY_EXAMS = 2 # Maximum gap, in months, that a patient can have on it's monthly exams.
MAX_PERIOD_TOLERANCE_FOR_OTHER_EXAMS = 1 # Maximum gap, in PERIODS, that a patient can have on it's other exams.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

db_raw = pd.read_excel('DBRaw.xlsx', index_col=0)

In [5]:
db_raw["Idade"].tolist()

[48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 31,
 31,
 31,
 31,
 32,
 32,
 32,
 32,
 32,
 32,
 33,
 33,
 33,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 43,
 43,
 43,
 43,
 43,
 43,
 43,
 43,
 43,
 43,
 43,
 43,
 44,


In [3]:
db_clean = pd.DataFrame()
db_completed = pd.DataFrame()
db_garbage = pd.DataFrame()

## New Rules

1. ⁠ ⁠Age >= 18 and number of records >= 3. -> Minimum requirements for analysis
2. ⁠ ⁠⁠Patient must have all of the exams in sequential order. The largest gap in this sequence must be **within the tolerance**.
3. ⁠ Patient must not have any monthly exams missing more than then tolerance.
4.  Patient must not have any quarterly, semianual or anual exams missing more than the period tolerace (a period can be every three, six or twelve months - depending on the exam)

In [21]:
from dateutil.relativedelta import relativedelta
exam_frequencies = {
    'Índice de Massa Corporal' : 1,
    'Taxa de Redução da Uréia' : 1,
    'HBs Ag' : 1,
    'K+' : 1,
    'Na+' : 1,
    'Hb do mês atual' : 1,
    'Ca Total do mês atual' : 1,
    'P' : 1,
    'Fosfatase Alcalina Total' : 3,
    'Último resultado do PTHi' : 3,
    'Alb.\n(Verde bromocresol)' : 3,
    'Ferritina' : 3,
    'Saturação de Transferrina' : 3,
    'Proteina Total' : 3,
    'Anti-HBs (Titulação)' : 6,
    'Anti-HBs' : 6,
    'Anti-HIV' : 12,
    'Colesterol Total' : 12,
}

def get_frequency(exam):
    match (exam_frequencies[exam]):
        case 1:
            return 'monthly'
        case 3:
            return 'quarterly'
        case 6:
            return 'semiannual'
        case 12:
            return 'annual'

def meets_minimum_criteria(patient_data):
    return len(patient_data) >= 3 and patient_data.iloc[0]['Idade'] >= 18

def is_sequential(patient_data):
    patient_data = patient_data.sort_values(by='Mês/ano')
    date_diffs = (patient_data['Mês/ano'].diff().diff() / pd.Timedelta(days=30)).dropna()
    return all(diff <= MAX_SEQUENCE_BREAK_TOLERANCE for diff in date_diffs)

def get_exam_completion_status(patient_data):
    oldest_completed_entry_index = get_first_completed_row_index(patient_data)
    if oldest_completed_entry_index == -1: return 'No row with all exams completed'

    patient_data = patient_data.iloc[oldest_completed_entry_index:]

    for exam, frequency in exam_frequencies.items():
        exam_data = patient_data[patient_data[exam].notna()]
        exam_data = exam_data.sort_values(by='Mês/ano')
        date_diffs = (exam_data['Mês/ano'].diff() / pd.Timedelta(days=30)).dropna()

        if frequency == 1:
            if any(diff > (1 + MAX_PERIOD_TOLERANCE_FOR_MONTHLY_EXAMS) for diff in date_diffs):
                return f'Uncompletable {get_frequency(exam)} exam'
        else:
            if any(diff > (frequency * (1 + MAX_PERIOD_TOLERANCE_FOR_OTHER_EXAMS)) for diff in date_diffs):
                return f'Uncompletable {get_frequency(exam)} exam'

    return 'Completable'

def get_first_completed_row_index(patient_data):
    oldest_completed_entry = None
    oldest_completed_entry_index = 0

    for index, row in patient_data.iterrows():
        if not row.isna().any():
            oldest_completed_entry = row
            break

        oldest_completed_entry_index += 1

    if oldest_completed_entry is None: return -1
    return oldest_completed_entry_index



Below we apply the functions to each patient, in order to determine if they fail on any of the rules, and if so, on which. Afterwards, we save the rules that were broken by the patient, and it's destination DB

In [22]:
patient_codes = db_raw['Código'].unique()
patients = []

completion_results = {}

for patient_code in patient_codes:
    broken_rule = ''
    patient_data = db_raw[db_raw['Código'] == str(patient_code)]

    if not meets_minimum_criteria(patient_data):
        broken_rule = 'Does not meet minimum criteria'
        patients.append({'Código': patient_code, 'Destino': 'DBGarbage', 'Regra quebrada': broken_rule})
        continue

    if not is_sequential(patient_data):
        broken_rule = 'Not sequential (exceeds two months)'
        patients.append({'Código': patient_code, 'Destino': 'DBGarbage', 'Regra quebrada': broken_rule})
        continue

    has_missing_data = patient_data.isna().values.any()

    if not has_missing_data:
        broken_rule = 'Missing data'
        patients.append({'Código': patient_code, 'Destino': 'DBClean', 'Regra quebrada': 'Nothing'})
        continue

    completion_status = get_exam_completion_status(patient_data)

    if completion_status != 'Completable':
        patients.append({'Código': patient_code, 'Destino': 'DBGarbage', 'Regra quebrada': completion_status})
        continue

    patients.append({'Código': patient_code, 'Destino': 'DBCompleted', 'Regra quebrada': 'Nothing'})


patients = pd.DataFrame(patients)

In [24]:
print(f'Number of patients: {len(patient_codes)}')
print(f'Number of patients in DBGarbage: {len(patients[patients["Destino"] == "DBGarbage"])}')
print(f'Number of patients in DBClean: {len(patients[patients["Destino"] == "DBClean"])}')
print(f'Number of patients in DBCompleted: {len(patients[patients["Destino"] == "DBCompleted"])}')

print()

print(patients["Destino"].value_counts())

print()

print(patients["Regra quebrada"].value_counts())

patients.to_excel('out/patients.xlsx', index=False)

Number of patients: 9367
Number of patients in DBGarbage: 4988
Number of patients in DBClean: 385
Number of patients in DBCompleted: 3994

Destino
DBGarbage      4988
DBCompleted    3994
DBClean         385
Name: count, dtype: int64

Regra quebrada
Nothing                                4379
No row with all exams completed        2898
Does not meet minimum criteria         1451
Not sequential (exceeds two months)     374
Uncompletable monthly exam              241
Uncompletable quarterly exam             18
Uncompletable semiannual exam             5
Uncompletable annual exam                 1
Name: count, dtype: int64
